In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
#from google.colab.patches import cv2_imshow

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [52]:
#parent_path = "/content/drive/MyDrive/Colab projects/signature-forgery-detection/signatures"
parent_path = os.path.join(str(globals()['_dh'][0]).replace("notebooks", ""), "signatures")
classes = ["full_forg", "full_org"]

In [53]:
def pre_process(img):

  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  blurred = cv2.GaussianBlur(gray, (5, 5), 0)
  _ ,th_img = cv2.threshold(blurred,0,255,cv2.THRESH_BINARY + cv2.THRESH_OTSU)

  return th_img

In [54]:
images_array = []
targets_array = []

for cls in classes:

  print(f"Loading class {cls}")
  curr_path = os.path.join(parent_path, cls)
  print(curr_path)

  img_path = os.path.join(curr_path, cls) + "1"
  print(img_path)

  for img in os.listdir(img_path):
    if(img.endswith(".png")):
      read_img = cv2.imread(os.path.join(img_path, img))
      pre_processed = pre_process(read_img)
      images_array.append(pre_processed)
      targets_array.append(classes.index(cls))

  print(f"{cls} class loaded successfully")

Loading class full_forg
/home/matteo/Repos/signature-forgery-detection/signatures/full_forg
/home/matteo/Repos/signature-forgery-detection/signatures/full_forg/full_forg1
full_forg class loaded successfully
Loading class full_org
/home/matteo/Repos/signature-forgery-detection/signatures/full_org
/home/matteo/Repos/signature-forgery-detection/signatures/full_org/full_org1
full_org class loaded successfully


In [21]:
descriptors = []
sift = cv2.SIFT_create()

max_len = 0

for img in images_array:
  _, desc = sift.detectAndCompute(img, None)
  np_desc = np.array(desc).flatten()
  # print(np_desc.shape)
  descriptors.append(np_desc)
  if len(np_desc) > max_len:
    max_len = len(np_desc)

descriptors = np.array(descriptors, dtype = object).flatten()
# print(descriptors.shape)
targets_array = np.array(targets_array, dtype = int).flatten()
# print(targets_array.shape)

In [26]:
padded_descriptors = np.empty([descriptors.size, max_len], dtype = object)
# print(padded_descriptors.shape)

for i, descriptor in enumerate(descriptors):
  padded = np.pad(descriptor, (0, max_len - len(descriptor))).flatten()
  padded_descriptors[i] = padded

# print(padded_descriptors.shape)
# print(str(padded_descriptors))


(48, 24064)
[[0.0 0.0 0.0 ... 0.0 0.0 0.0]
 [0.0 0.0 0.0 ... 0.0 0.0 0.0]
 [0.0 4.0 37.0 ... 0.0 0.0 0.0]
 ...
 [0.0 0.0 0.0 ... 0.0 0.0 0.0]
 [0.0 0.0 0.0 ... 0.0 0.0 0.0]
 [0.0 0.0 0.0 ... 0.0 0.0 0.0]]


In [23]:
X = padded_descriptors
y = targets_array

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state = 42)

In [24]:
svm = SVC(kernel = "linear", probability = True)

svm.fit(X_train, y_train)

SVC(kernel='linear', probability=True)

In [25]:
y_pred = svm.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.7333333333333333
